In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Feb  9 19:20:51 2022

@author: RRZ
"""
from joblib import dump
import math
import pandas as pd
import numpy as np
import random
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

df_raw_train = pd.read_csv('../data/raw/train.csv')
df_raw_test = pd.read_csv('../data/raw/test.csv')

train_df_size = 0.8
rand_state_ind = 42
validation_df_size = 0.2

df_cleaned_train = df_raw_train.copy()

target = df_cleaned_train.pop('TARGET_5Yrs')
IDlist_train = df_cleaned_train.pop('Id')
df_col_names = df_cleaned_train.columns
train_dataset_size = IDlist_train.size

scaler = StandardScaler()
array_cleaned_train = scaler.fit_transform(df_cleaned_train)
df_cleaned2_train = pd.DataFrame(array_cleaned_train,columns=df_col_names)
#df_cleaned2_train.insert(loc=0,column='Id',value=IDlist_train)
#df_cleaned2_train.set_index(keys='Id',drop=False,verify_integrity=True)

df_cleaned_test = df_raw_test.copy()
IDlist_test = df_cleaned_test.pop('Id')
array_cleaned_test = scaler.transform(df_cleaned_test) 
df_cleaned2_test = pd.DataFrame(array_cleaned_test,columns=df_col_names)
#df_cleaned2_test.insert(loc=0,column='Id',value=IDlist_test)

from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2)
poly.feature_names_in_ = df_col_names
array_clean_poly_train = poly.fit_transform(df_cleaned_train)
polyFeatureNames = poly.get_feature_names_out()
df_cleaned_poly_train = pd.DataFrame(array_clean_poly_train,columns=polyFeatureNames)
#df_cleaned_poly_train.insert(loc=0,column='Id',value=IDlist_train)

array_clean_poly_test = poly.transform(df_cleaned_test)
df_cleaned_poly_test = pd.DataFrame(array_clean_poly_test,columns=polyFeatureNames)
#df_cleaned_poly_test.insert(loc=0,column='Id',value=IDlist_test)

X_data, X_val, y_data, y_val = train_test_split(df_cleaned2_train, target, train_size=train_df_size, random_state=rand_state_ind, stratify=target)

y_train_pos_count = sum(target)
y_train_neg_count = target.size - y_train_pos_count

y_train_pos_count = sum(y_data)
y_train_neg_count = y_data.size - y_train_pos_count

y_val_pos_count = sum(y_val)
y_val_neg_count = y_val.size - y_val_pos_count

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

from sklearn.linear_model import LogisticRegression
logRegModel = LogisticRegression(penalty = 'l2')
clf = logRegModel.fit(X_data,y_data)
y_train_pred = clf.predict(X_data)
y_val_pred = clf.predict(X_val)
y_test_pred = clf.predict(df_cleaned2_test)

roc_train = roc_auc_score(y_data, clf.predict_proba(X_data)[:,1])
roc_val = roc_auc_score(y_val, clf.predict_proba(X_val)[:,1])

y_train_pred_pos_count = sum(y_train_pred)
y_train_pred_neg_count = y_data.size - y_train_pred_pos_count

y_val_pred_pos_count = sum(y_val_pred)
y_val_pred_neg_count = y_val.size - y_val_pred_pos_count

from sklearn.linear_model import SGDClassifier
SGDModel = SGDClassifier(loss='modified_huber')
clf2 = SGDModel.fit(X_data,y_data)
y_train_pred2 = clf2.predict(X_data)
y_val_pred2 = clf2.predict(X_val)
y_test_pred2 = clf2.predict(df_cleaned2_test)

roc_train2 = roc_auc_score(y_data, clf2.predict_proba(X_data)[:,1])
roc_val2 = roc_auc_score(y_val, clf2.predict_proba(X_val)[:,1])

y_train_pred_pos_count2 = sum(y_train_pred2)
y_train_pred_neg_count2 = y_data.size - y_train_pred_pos_count2

y_val_pred_pos_count2 = sum(y_val_pred2)
y_val_pred_neg_count2 = y_val.size - y_val_pred_pos_count2

from sklearn.ensemble import RandomForestClassifier
RFModel = RandomForestClassifier()
clf3 = RFModel.fit(X_data,y_data)
y_train_pred3 = clf3.predict(X_data)
y_val_pred3 = clf3.predict(X_val)
y_test_pred3 = clf3.predict(df_cleaned2_test)

roc_train3 = roc_auc_score(y_data, clf3.predict_proba(X_data)[:,1])
roc_val3 = roc_auc_score(y_val, clf3.predict_proba(X_val)[:,1])

y_train_pred_pos_count3 = sum(y_train_pred3)
y_train_pred_neg_count3 = y_data.size - y_train_pred_pos_count3

y_val_pred_pos_count3 = sum(y_val_pred3)
y_val_pred_neg_count3 = y_val.size - y_val_pred_pos_count3


X_data_poly, X_val_poly, y_data_poly, y_val_poly = train_test_split(df_cleaned_poly_train, target, train_size=train_df_size, random_state=rand_state_ind, stratify=target)

clf_poly = logRegModel.fit(X_data_poly,y_data_poly)
y_train_poly_pred = clf_poly.predict(X_data_poly)
y_val_poly_pred = clf_poly.predict(X_val_poly)
y_test_poly_pred = clf_poly.predict(df_cleaned_poly_test)

roc_train_poly = roc_auc_score(y_data_poly, clf_poly.predict_proba(X_data_poly)[:,1])
roc_val_poly = roc_auc_score(y_val_poly, clf_poly.predict_proba(X_val_poly)[:,1])

clf_poly2 = SGDModel.fit(X_data_poly,y_data_poly)
y_train_poly_pred2 = clf_poly2.predict(X_data_poly)
y_val_poly_pred2 = clf_poly2.predict(X_val_poly)
y_test_poly_pred2 = clf_poly2.predict(df_cleaned_poly_test)

roc_train_poly2 = roc_auc_score(y_data_poly, clf_poly2.predict_proba(X_data_poly)[:,1])
roc_val_poly2 = roc_auc_score(y_val_poly, clf_poly2.predict_proba(X_val_poly)[:,1])

clf_poly3 = RFModel.fit(X_data_poly,y_data_poly)
y_train_poly_pred3 = clf_poly3.predict(X_data_poly)
y_val_poly_pred3 = clf_poly3.predict(X_val_poly)
y_test_poly_pred3 = clf_poly3.predict(df_cleaned_poly_test)

roc_train_poly3 = roc_auc_score(y_data_poly, clf_poly3.predict_proba(X_data_poly)[:,1])
roc_val_poly3 = roc_auc_score(y_val_poly, clf_poly3.predict_proba(X_val_poly)[:,1])


def BuildCSVforSubmission(IDlist,PredictionList):
    iErrCode = 0
    fFileName = 'submission.csv'
    if (IDlist.size != 3799) or (PredictionList.size != 3799):
        iErrCode = 1
    if iErrCode == 0:
        if not(isinstance(IDlist, pd.Series)):
            list1 = pd.Series(IDlist)
        else:
            list1 = IDlist.copy()
        if not(isinstance(PredictionList, pd.Series)):
            list2 = pd.Series(PredictionList)
        else:
            list2 = IDlist.copy()
        #list2.round(2)
        #df_out = pd.DataFrame([list1,list2])
        df_out = pd.DataFrame({'Id':list1,'TARGET_5Yrs':list2})
        df_out.to_csv(path_or_buf = '../data/processed/'+fFileName,index=False)
    return iErrCode

roc_test = clf.predict_proba(df_cleaned2_test)[:,1]
code = BuildCSVforSubmission(IDlist_test,roc_test)

from sklearn.model_selection import cross_validate
from sklearn import metrics

#sorted(metrics.SCORERS.keys())

scoring = ['roc_auc']
#logreg_scores2 = cross_validate(logRegModel, df_cleaned2_train, target, scoring=scoring, cv=20, return_train_score=True,return_estimator=True)
logreg_scores = cross_validate(logRegModel, df_cleaned2_train, target, scoring=scoring, cv=20, return_train_score=True,return_estimator=True)
SDG_scores = cross_validate(SGDModel, df_cleaned2_train, target, scoring=scoring, cv=20, return_train_score=True,return_estimator=True)
randfor_scores = cross_validate(RFModel, df_cleaned2_train, target, scoring=scoring, cv=20, return_train_score=True,return_estimator=True)

best_score = [np.max(logreg_scores.get('test_roc_auc')),np.max(SDG_scores.get('test_roc_auc')),np.max(randfor_scores.get('test_roc_auc'))]
best_score_logreg = np.max(logreg_scores.get('test_roc_auc'))
logreg_cv_models = logreg_scores.get('estimator')
logreg_cv_scores = logreg_scores.get('test_roc_auc')
logreg_cv_best_model = np.take(logreg_cv_models,(logreg_cv_scores==best_score_logreg))
roc_test_cv = logreg_cv_best_model[0].predict_proba(df_cleaned2_test)[:,1]

roc_test = clf.predict_proba(df_cleaned2_test)[:,1]
code = BuildCSVforSubmission(IDlist_test,roc_test_cv)

# best_score_logreg2 = np.max(logreg_scores2.get('test_roc_auc'))
# logreg_cv_models2 = logreg_scores2.get('estimator')
# logreg_cv_scores2 = logreg_scores2.get('test_roc_auc')
# logreg_cv_best_model2 = np.take(logreg_cv_models2,(logreg_cv_scores2==best_score_logreg2))
# roc_test_cv2 = logreg_cv_best_model2[0].predict_proba(df_cleaned2_test)[:,1]


from sklearn.neural_network import MLPClassifier
NNModel = MLPClassifier()
clf4 = NNModel.fit(X_data,y_data)
y_train_pred4 = clf4.predict(X_data)
y_val_pred4 = clf4.predict(X_val)
y_test_pred4 = clf4.predict(df_cleaned2_test)

roc_train4 = roc_auc_score(y_data, clf4.predict_proba(X_data)[:,1])
roc_val4 = roc_auc_score(y_val, clf4.predict_proba(X_val)[:,1])

NN_scores = cross_validate(NNModel, df_cleaned2_train, target, scoring=scoring, cv=20, return_train_score=True,return_estimator=True)
best_score_NN = np.max(NN_scores.get('test_roc_auc'))

from sklearn.model_selection import GridSearchCV
logRegModel = LogisticRegression(penalty = 'l2',max_iter=10000,random_state=42)
models = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
cRegStr = np.logspace(start=1e-3,stop=1e3,num=7)
cRegStr = list([1.0,10])
class_weight = [None,'balanced']
scoring = 'roc_auc'
param_grid = {'C':cRegStr,'class_weight':class_weight,'solver':models}
#param_grid = {'C':cRegStr,'solver':models}
cv = 10

clf11 = GridSearchCV(estimator=logRegModel,param_grid=param_grid,cv=cv,scoring=scoring,return_train_score=True,verbose=3)
clf11.fit(df_cleaned2_train, target)
clf11.best_estimator_
clf11.score(df_cleaned2_train, target)
clf11.cv_results_.keys()
test_grid_1 = clf11.predict_proba(df_cleaned2_test)[:,1]

dump(clf11,  '../models/logreg_10cv_gridsearch_basic.joblib')

logRegModel = LogisticRegression(penalty = 'l2',max_iter=10000,random_state=42)
models = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
cRegStr = np.logspace(start=-3,stop=3,num=7)
#cRegStr = list([1.0,10])
class_weight = [None,'balanced']
scoring = 'roc_auc'
param_grid1a = {'C':cRegStr,'class_weight':class_weight,'solver':models}
#param_grid = {'C':cRegStr,'solver':models}
cv = 10

clf11a = GridSearchCV(estimator=logRegModel,param_grid=param_grid1a,cv=cv,scoring=scoring,return_train_score=True,verbose=3)
clf11a.fit(df_cleaned2_train, target)
clf11a.best_estimator_
clf11a.score(df_cleaned2_train, target)
clf11a.cv_results_.keys()
test_grid_1a = clf11a.predict_proba(df_cleaned2_test)[:,1]

dump(clf11a,  '../models/logreg_10cv_gridsearch_basic2.joblib')


RFModel = RandomForestClassifier(random_state=42)
n_estimators = np.logspace(start=1, stop=round(math.log(train_dataset_size/2,2)),base=2,num=5).astype(int)
max_depth = np.arange(start=2,stop=5)
min_samples_split = np.logspace(start=1, stop=round(math.log(train_dataset_size/2),2),base=2,num=5).astype(int)
param_grid2 = {'max_depth':max_depth,'n_estimators':n_estimators,'min_samples_split':min_samples_split}

clf12 = GridSearchCV(estimator=RFModel,param_grid=param_grid2,cv=cv,scoring=scoring,return_train_score=True,verbose=3)
clf12.fit(df_cleaned2_train, target)
clf12.best_estimator_
clf12.score(df_cleaned2_train, target)
clf12.cv_results_.keys()
test_grid_2 = clf12.predict_proba(df_cleaned2_test)[:,1]

dump(clf12,  '../models/randforest_10cv_gridsearch_basic.joblib')

SGDModel = SGDClassifier(max_iter=10000,random_state=42)
loss_fn = ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron', 'squared_error', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive']
alpha = np.linspace(start=-4, stop=3, num=8)
l1_ratio = np.linspace(start=0, stop=1, num=6)
learning_rate = ['constant', 'optimal', 'invscaling', 'adaptive']
param_grid3 = {'loss':loss_fn,'alpha':alpha,'l1_ratio':l1_ratio,'learning_rate':learning_rate}

clf13 = GridSearchCV(estimator=SGDModel,param_grid=param_grid3,cv=cv,scoring=scoring,return_train_score=True,verbose=3)
clf13.fit(df_cleaned2_train, target)
clf13.best_estimator_
clf13.score(df_cleaned2_train, target)
clf13.cv_results_.keys()
test_grid_3 = clf13.predict_proba(df_cleaned2_test)[:,1]

dump(clf13,  '../models/SDG_10cv_gridsearch_basic.joblib')

from sklearn.utils import resample
df_train_posclass = df_raw_train[df_raw_train['TARGET_5Yrs']==1]
df_train_negclass = df_raw_train[df_raw_train['TARGET_5Yrs']==0]
df_train_negclass_upsampled = resample(df_train_negclass, replace=True, n_samples=y_train_pos_count, random_state=42)

df_cleaned3_train = pd.concat([df_train_posclass,df_train_negclass_upsampled])

target2 = df_cleaned3_train.pop('TARGET_5Yrs')
IDlist_train = df_cleaned3_train.pop('Id')

clf14 = GridSearchCV(estimator=logRegModel,param_grid=param_grid1a,cv=cv,scoring=scoring,return_train_score=True,verbose=3)
clf14.fit(df_cleaned3_train, target2)
clf14.best_estimator_
clf14.score(df_cleaned3_train, target2)
clf14.cv_results_.keys()
test_grid_4 = clf11.predict_proba(df_cleaned2_test)[:,1]

dump(clf14,  '../models/logreg_10cv_gridsearch_upsampled.joblib')

clf15 = GridSearchCV(estimator=RFModel,param_grid=param_grid2,cv=cv,scoring=scoring,return_train_score=True,verbose=3)
clf15.fit(df_cleaned3_train, target2)
clf15.best_estimator_
clf15.score(df_cleaned3_train, target2)
clf15.cv_results_.keys()
test_grid_5 = clf11.predict_proba(df_cleaned2_test)[:,1]

dump(clf15,  '../models/randforest_10cv_gridsearch_upsampled.joblib')

clf16 = GridSearchCV(estimator=SGDModel,param_grid=param_grid3,cv=cv,scoring=scoring,return_train_score=True,verbose=3)
clf16.fit(df_cleaned3_train, target2)
clf16.best_estimator_
clf16.score(df_cleaned3_train, target2)
clf16.cv_results_.keys()
test_grid_6 = clf11.predict_proba(df_cleaned2_test)[:,1]

dump(clf16,  '../models/SDG_10cv_gridsearch_upsampled.joblib')

import xgboost as xgb
data_dmatrix_train = xgb.DMatrix(data=df_cleaned3_train,label=target)
data_dmatrix_test = xgb.DMatrix(data=df_cleaned2_test)

XGBModel = xgb.XGBClassifier(use_label_encoder=False, objective='binary:logistic', eval_metric='auc')

X_data, X_val, y_data, y_val = train_test_split(df_cleaned3_train, target, train_size=train_df_size, random_state=rand_state_ind, stratify=target)

clf6 = XGBModel.fit(X_data, y_data)
y_pred = XGBModel.predict_proba(df_cleaned2_test)[:,1]
roc_train_XGB = roc_auc_score(y_data, clf6.predict_proba(X_data)[:,1])
roc_val_XGB = roc_auc_score(y_val, clf6.predict_proba(X_val)[:,1])